In [232]:
import pandas as pd
import numpy as np

In [233]:
df = pd.read_csv('All orders - short custom description.csv', low_memory=False)
df.drop_duplicates(inplace=True)
df['customs_description'] = df['customs_description'].astype(str)
df['F_Cat'] = df['F_Cat'].astype(str)
df['quantity'] = df['quantity'].astype(int)

In [234]:
import matplotlib
list_colours = []
for name, hex in matplotlib.colors.cnames.items():
    name = name.title()
    list_colours.append(name)

# list_colours = []
colours_df = pd.read_csv('colours.csv')
colours_df['colours'] = colours_df['colours'].str.strip()
for j in colours_df['colours'].unique().tolist():
    list_colours.append(j)

for i in list_colours:
    i = i.title()

list_colours = list(dict.fromkeys(list_colours))

df['colour'] = df['title'].str.findall(r'(?i)\b(?:{})\b'.format('|'.join(list_colours))).apply(', '.join)

df2 = df['colour'].str.split(',', expand=True)
df['colour'] = df2[0]

In [235]:
orig_df = df.copy()

In [236]:
df = df[(df['date'] >= '2024-03-01') & (df['date'] <= '2024-03-31')]
df = df[df['Channel'] == 'Leather Company']

In [237]:
df = df[(df['customs_description'] == 'Ladies > Leather Jackets')]
df = df[df['model'] != '5 pads set']

temp_dispatched = df[(df['order_state'] == 'Order Dispatched') | (df['order_state'] == 'Order Refunded')]
temp_refunded = df[df['order_state'] == 'Order Refunded']

df['model'] = df['model'].fillna("(No assigned category)")
df['colour'] = df['colour'].fillna("(No assigned colour)")
df['attribute_summary'] = df['attribute_summary'].fillna("(No assigned size)")

temp_dispatched = temp_dispatched.groupby(['customs_description', 'model', 'colour', 'attribute_summary']).sum()
temp_dispatched.rename(columns={'quantity': 'Units Sold', 'price_inc': 'Total Revenue (£)'}, inplace=True)

temp_refunded = temp_refunded.groupby(['customs_description', 'model', 'colour', 'attribute_summary']).sum()
temp_refunded.rename(columns={'quantity': 'Units Refunded', 'price_inc': 'Total Refund (£)'}, inplace=True)

temp_ladies = pd.concat([temp_dispatched, temp_refunded], axis=1)
temp_ladies = temp_ladies[['Units Sold', 'Total Revenue (£)', 'Units Refunded', 'Total Refund (£)']]
temp_ladies['Units Sold'] = temp_ladies['Units Sold'].fillna(0)
temp_ladies['Total Revenue (£)'] = temp_ladies['Total Revenue (£)'].fillna(0)
temp_ladies['Units Refunded'] = temp_ladies['Units Refunded'].fillna(0)
temp_ladies['Total Refund (£)'] = temp_ladies['Total Refund (£)'].fillna(0)
temp_ladies['Final Revenue (£)'] = temp_ladies['Total Revenue (£)'] - temp_ladies['Total Refund (£)']
temp_ladies.loc['Total', :] = temp_ladies.sum(numeric_only=True).values
temp_ladies['Units Sold'] = temp_ladies['Units Sold'].astype(int)
temp_ladies['Units Refunded'] = temp_ladies['Units Refunded'].astype(int)
temp_ladies['Total Revenue (£)'] = temp_ladies['Total Revenue (£)'].astype(int)
temp_ladies['Total Refund (£)'] = temp_ladies['Total Refund (£)'].astype(int)
temp_ladies['Final Revenue (£)'] = temp_ladies['Final Revenue (£)'].astype(int)

In [238]:
df = orig_df[(orig_df['date'] >= '2024-02-01') & (orig_df['date'] <= '2024-02-29')]
df = df[df['Channel'] == 'Leather Company']

In [239]:
df = df[(df['customs_description'] == 'Mens > Leather Jackets')]

temp_dispatched = df[(df['order_state'] == 'Order Dispatched') | (df['order_state'] == 'Order Refunded')]
temp_refunded = df[df['order_state'] == 'Order Refunded']

df['model'] = df['model'].fillna("(No assigned category)")
df['colour'] = df['colour'].fillna("(No assigned colour)")
df['attribute_summary'] = df['attribute_summary'].fillna("(No assigned size)")

temp_dispatched = temp_dispatched.groupby(['customs_description', 'model', 'colour', 'attribute_summary']).sum()
temp_dispatched.rename(columns={'quantity': 'Units Sold', 'price_inc': 'Total Revenue (£)'}, inplace=True)

temp_refunded = temp_refunded.groupby(['customs_description', 'model', 'colour', 'attribute_summary']).sum()
temp_refunded.rename(columns={'quantity': 'Units Refunded', 'price_inc': 'Total Refund (£)'}, inplace=True)

temp = pd.concat([temp_dispatched, temp_refunded], axis=1)
temp = temp[['Units Sold', 'Total Revenue (£)', 'Units Refunded', 'Total Refund (£)']]
temp['Units Sold'] = temp['Units Sold'].fillna(0)
temp['Total Revenue (£)'] = temp['Total Revenue (£)'].fillna(0)
temp['Units Refunded'] = temp['Units Refunded'].fillna(0)
temp['Total Refund (£)'] = temp['Total Refund (£)'].fillna(0)
temp['Final Revenue (£)'] = temp['Total Revenue (£)'] - temp['Total Refund (£)']
temp.loc['Total', :] = temp.sum(numeric_only=True).values
temp['Units Sold'] = temp['Units Sold'].astype(int)
temp['Units Refunded'] = temp['Units Refunded'].astype(int)
temp['Total Revenue (£)'] = temp['Total Revenue (£)'].astype(int)
temp['Total Refund (£)'] = temp['Total Refund (£)'].astype(int)
temp['Final Revenue (£)'] = temp['Final Revenue (£)'].astype(int)

In [240]:
temp_ladies

Units Sold  \
customs_description      model       colour   attribute_summary               
Ladies > Leather Jackets 1127        Red      Size: 12                    1   
                         1132 BIKER  Black    Size: 14                    1   
                         1132 biker  Tan      Size: 22                    1   
                         1141        Beige    Size: 10                    1   
                                              Size: 12                    1   
...                                                                     ...   
                         moto bomber Burgundy Size: 18                    1   
                         nirma       Black    Size: 8                     1   
                                     Grey     Size: 10                    1   
                                              Size: 16                    1   
Total                                                                   124   

                                                                 Total Revenue (£)  \
customs_description      model       colour   attribute_summary                      
Ladies > Leather Jackets 1127        Red      Size: 12                         129   
                         1132 BIKER  Black    Size: 14                         165   
                         1132 biker  Tan      Size: 22                         199   
                         1141        Beige    Size: 10                         129   
                                              Size: 12                         129   
...                                                                            ...   
                         moto bomber Burgundy Size: 18                         169   
                         nirma       Black    Size: 8                          129   
                                     Grey     Size: 10                         129   
                                              Size: 16                         129   
Total                                                                        15515   

                                                                 Units Refunded  \
customs_description      model       colour   attribute_summary                   
Ladies > Leather Jackets 1127        Red      Size: 12                        0   
                         1132 BIKER  Black    Size: 14                        0   
                         1132 biker  Tan      Size: 22                        0   
                         1141        Beige    Size: 10                        1   
                                              Size: 12                        1   
...                                                                         ...   
                         moto bomber Burgundy Size: 18                        1   
                         nirma       Black    Size: 8                         0   
                                     Grey     Size: 10                        0   
                                              Size: 16                        1   
Total                                                                        34   

                                                                 Total Refund (£)  \
customs_description      model       colour   attribute_summary                     
Ladies > Leather Jackets 1127        Red      Size: 12                          0   
                         1132 BIKER  Black    Size: 14                          0   
                         1132 biker  Tan      Size: 22                          0   
                         1141        Beige    Size: 10                        129   
                                              Size: 12                        129   
...                                                                           ...   
                         moto bomber Burgundy Size: 18                        169   
                         nirma       Black    Size: 8                    

In [241]:
temp

Units Sold  \
customs_description    model        colour attribute_summary               
Mens > Leather Jackets 1895         Black  Size: 2XL                   2   
                                           Size: 4XL                   1   
                                           Size: 5XL                   1   
                                           Size: 6XL                   1   
                                           Size: M                     1   
...                                                                  ...   
                       stefan biker White  Size: L                     1   
                       super bomber Black  Size: 2XL                   1   
                                           Size: M                     1   
                                           Size: XL                    2   
Total                                                                129   

                                                              Total Revenue (£)  \
customs_description    model        colour attribute_summary                      
Mens > Leather Jackets 1895         Black  Size: 2XL                        273   
                                           Size: 4XL                        157   
                                           Size: 5XL                        199   
                                           Size: 6XL                        220   
                                           Size: M                          149   
...                                                                         ...   
                       stefan biker White  Size: L                          165   
                       super bomber Black  Size: 2XL                        165   
                                           Size: M                          199   
                                           Size: XL                         398   
Total                                                                     26569   

                                                              Units Refunded  \
customs_description    model        colour attribute_summary                   
Mens > Leather Jackets 1895         Black  Size: 2XL                       1   
                                           Size: 4XL                       0   
                                           Size: 5XL                       1   
                                           Size: 6XL                       1   
                                           Size: M                         0   
...                                                                      ...   
                       stefan biker White  Size: L                         0   
                       super bomber Black  Size: 2XL                       0   
                                           Size: M                         0   
                                           Size: XL                        0   
Total                                                                     21   

                                                              Total Refund (£)  \
customs_description    model        colour attribute_summary                     
Mens > Leather Jackets 1895         Black  Size: 2XL                       149   
                                           Size: 4XL                         0   
                                           Size: 5XL                       199   
                                           Size: 6XL                       220   
                                           Size: M                           0   
...                                                                        ...   
                       stefan biker White  Size: L                           0   
                       super bomber Black  Size: 2XL                         0   
                                           Size: M                           0   
                                           Size: X

In [242]:
final = pd.concat([temp_ladies, temp])

In [243]:
final.to_excel('Leather Company - Mar 2024 - Leather Jackets.xlsx')

In [244]:
# # Leather Company
# import matplotlib.pyplot as plt

# temp_dispatched = temp_dispatched[temp_dispatched['price_inc'] < 400]
# temp_refunded = temp_refunded[temp_refunded['price_inc'] < 400]

# fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))
# fig.suptitle('Leather Company - Ladies Leather Jackets')
# plt.figure(figsize=(20,6))
# bins = [109, 119, 129, 139, 149, 159, 169, 179]
# ax1.hist(temp_dispatched['price_inc'], bins=10, edgecolor="k")
# ax1.set_title('Sold')
# ax2.hist(temp_refunded['price_inc'], bins=10, edgecolor="k")
# ax2.set_title('Refunded')
# y_bins = [0, 10, 20, 30, 40, 50]
# ax1.set_yticks(y_bins)
# ax2.set_yticks(y_bins)
# # ax1.set_xticks(bins)
# # ax2.set_xticks(bins)
# plt.show()